In [41]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools

# タイプ

最初に行うべきことは、あなたの問題に適したタイプを考えることです。次に、使用可能なタイプのリストを調べる代わりに、DEAPを使用して独自のビルドを作成することができます。これはcreatorモジュールで行われます。適切なタイプを作成することは圧倒的に思えるかもしれませんが、作成者はそれを非常に簡単にします。実際、これは通常1行で行われます。たとえば、次の例ではFitnessMin、最小化問題のIndividualクラスと、作成したばかりのフィットネスに設定されたフィットネス属性を持つリストから派生したクラスを作成します。



In [26]:
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


# 初期化

型が作成されたら、それらを時々ランダムな値で埋める必要があります。再び、DEAPはそれを行うための簡単なメカニズムを提供します。これToolboxは、必要なことをすることができるイニシャライザを含むあらゆる種類のツール用のコンテナです。次のコード行は、ランダムな浮動小数点数を含む個体とそれを含む個体の初期化子を作成するコードの最後の行です。

In [27]:
import random
from deap import tools

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

これは、ランダムな浮動小数点数で初期化された個体から母集団を初期化する関数を作成します。関数は、指定された名前の下にデフォルトの引数でツールボックスに登録されます。たとえば、関数toolbox.population()を呼び出して、即座に母集団を作成することができます。その他の初期化メソッドは、「型の作成」チュートリアルとさまざまな 例にあります。



# 演算子

演算子は初期化子と似ていますが、toolsモジュールの中には既に実装されているものもあります。完璧なものを選んだら、ツールボックスに登録するだけです。さらに、評価関数を作成する必要があります。これがDEAPで行われる方法です。

In [28]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

登録された関数は、ツールボックスによって名前が変更され、演算子名に依存しない汎用アルゴリズムが可能になります。適性値は反復可能でなければならないことに注意してください。それは評価関数でタプルを返す理由です。この上のその他の演算子とアルゴリズムの チュートリアルと例。

# アルゴリズム

In [34]:
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

In [35]:
if __name__ == "__main__":
    print(main())

[[-2.526122668581078, -2.4915941203864675, -0.8665735192538824, -3.843198455247002, -5.222751045261218, -1.2737860368451739, -1.536234059556639, -3.151203304975172, -4.6559845088634075, -2.4514178349252123], [-2.526122668581078, -2.8437787862628388, -0.8665735192538824, -3.843198455247002, -5.222751045261218, -1.7039036618265355, -1.536234059556639, -2.2764850857057986, -4.6559845088634075, -2.4514178349252123], [-2.3030997561656945, -2.4915941203864675, -1.707303386843611, -3.843198455247002, -5.222751045261218, -1.2737860368451739, -1.536234059556639, -2.2764850857057986, -3.995133734793293, -2.069258184693871], [-1.307613014847673, -2.8437787862628388, -0.8665735192538824, -3.843198455247002, -5.222751045261218, -1.7039036618265355, -1.536234059556639, -2.2764850857057986, -4.6559845088634075, -2.4514178349252123], [-2.526122668581078, -2.8437787862628388, -0.8665735192538824, -3.843198455247002, -5.222751045261218, -1.2737860368451739, -1.536234059556639, -3.151203304975172, -4.655

# タイプの作成

## 適応度

In [36]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [38]:
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))

/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


## 個体

## フロートのリスト

In [39]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [44]:
creator.create("Individual", array, typecode="d", fitness=creator.FitnessMax)
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

NameError: name 'array' is not defined

## 順列 Permutation

In [45]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)


/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


## 算術式

In [46]:
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools

pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)

/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


# 演算子とアルゴリズム

## 最初の個体

In [47]:
import random

from deap import base
from deap import creator
from deap import tools

IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [48]:
ind1 = toolbox.individual()

print(ind1)               # [0.86..., 0.27..., 0.70..., 0.03..., 0.87...]
print(ind1.fitness.valid) # False

[0.093838437199392, 0.037988598965098985, 0.9391797298936297, 0.1654577597684821, 0.6640481733773015]
False


## 評価

In [51]:
def evaluate(individual):
    # Do some hard computing on the individual
    a = sum(individual)
    b = len(individual)
    return a, 1. / b

ind1.fitness.values = evaluate(ind1)
print(ind1.fitness.valid)    # True
print(ind1.fitness)          # (2.73, 0.2)

True
(1.900512699203904, 0.2)
